In [ ]:
#READ THIS:!!!!!!!!
#Read Real Stock Testing TODO List.md for more information on what to do next.

#REWRITE CODE FOR ALPACA API INSTEAD OF YFINANCE

import pandas as pd
import matplotlib.pyplot as plt
import warnings
from calculate import calculate_ema, calculate_sma, calculate_macd, calculate_rsi, calculate_dmi
from config import API_KEY, API_SECRET
from alpaca_trade_api.rest import REST, TimeFrame 
warnings.filterwarnings('ignore', category=UserWarning)

# SETTINGS
BASE_URL = "https://paper-api.alpaca.markets"  # Paper trading endpoint
api = REST(API_KEY, API_SECRET, BASE_URL)

tickers = input("Enter ticker symbols separated by commas (e.g., AAPL,MSFT): ").upper().split(',')
start_date = "2025-01-01"
end_date = "2025-10-01"

#User Input for Moving Average Window Sizes taken away for testing purposes
short_window = 12
long_window = 26
use_ema_input = input("Use EMA instead of SMA? (True/False): ").strip().lower()
use_ema = (use_ema_input in ['true', 't', 'yes', 'y', 'e'])

#User Input for Starting Cash and Shares per Trade taken away for testing purposes
starting_cash = 100000
shares_per_trade = 100

total_realized_profit = 0
ending_cash = starting_cash
all_results = []

def backtest_crossover(ticker):
    global total_realized_profit,ending_cash
   
    #Have to fix below code to work with Alpaca API
    print(f"\nDownloading data for {ticker}...")
    data = api.get_bars(ticker, TimeFrame.Day, start=start_date, end=end_date).df
    df = data.copy()

    if data.empty:
        print(f"No data found for {ticker}. Skipping.\n")
        return

    prices = data['close'].squeeze()

    # Calculate S/E moving averages
    short_ma = calculate_ema(prices, short_window) if use_ema else calculate_sma(prices, short_window)
    long_ma = calculate_ema(prices, long_window) if use_ema else calculate_sma(prices, long_window)

    #Calculates MACD line, signal line, and histogram
    macd_line = calculate_macd(prices)
    signal_line = macd_line.ewm(span=9, adjust=False).mean()
    macd_hist = macd_line - signal_line

    # Calculates RSI
    rsiwindow = 14
    rsi = calculate_rsi(prices, window=rsiwindow)
        
    # RSI overbought/oversold levels
    rsi_overbought = 65
    rsi_oversold = 35

    #Directional Movement Index (DMI) system and, the Average Directional Index (ADX)
    period_window = 14
    df['+DM_smoothed'], df['-DM_smoothed'] = calculate_dmi(df, period=period_window)

    #DF version of DMI calculation - Testing Code
    #up_move = df['high'].diff()
    #down_move = df['low'].shift(1) - df['low']
    #df['+DM'] = up_move.where((up_move > down_move) & (up_move > 0), 0)
    #df['-DM'] = down_move.where((down_move > up_move) & (down_move > 0), 0)
    #df['+DM_smoothed'] = df['+DM'].ewm(alpha=1/14, adjust=False).mean()
    #df['-DM_smoothed'] = df['-DM'].ewm(alpha=1/14, adjust=False).mean()

    # Generate signals: Buy, Sell, Hold
    signals = ['Hold']
    for i in range(1, len(prices)):
        if short_ma.iloc[i] > long_ma.iloc[i] and short_ma.iloc[i-1] <= long_ma.iloc[i-1]:
            if rsi.iloc[i] < rsi_oversold: #or not use_rsi_fliter:  # Buy only if RSI indicates oversold
                signals.append('Strong Buy')
                print(f"ALERT: Strong Buy signal on {prices.index[i].date()} at price {prices.iloc[i]:.2f}")
            else:
                signals.append('Buy')

        elif short_ma.iloc[i] < long_ma.iloc[i] and short_ma.iloc[i-1] >= long_ma.iloc[i-1]:
            if rsi.iloc[i] > rsi_overbought: #or not use_rsi_fliter:  # Sell only if RSI indicates overbought
                signals.append('Strong Sell')
                print(f"ALERT: Strong Sell signal on {prices.index[i].date()} at price {prices.iloc[i]:.2f}")
            else:
                signals.append('Sell')
        else:
            signals.append('Hold')
    

    #Currently Lookback for RSI is in Days. Consider changing to minutes or Hours for more Day trading style
    lookback = 5    
    lookback_signals = ['Hold'] * len(prices)
    for i in range(lookback, len(prices)):
        recent_rsi = rsi.iloc[max(0, i-lookback):i+1]
        if rsi.iloc[i-1] >= rsi_oversold and rsi.iloc[i] < rsi_oversold:
            print(f"Lookback ALERT: Recent RSI indicates oversold on {prices.index[i].date()} at price {prices.iloc[i]:.2f}")
            lookback_signals[i]='Buy Lookback RSI Confirmed'
        if rsi.iloc[i-1] <= rsi_overbought and rsi.iloc[i] > rsi_overbought:
            print(f"Lookback ALERT: Recent RSI indicates overbought on {prices.index[i].date()} at price {prices.iloc[i]:.2f}")
            lookback_signals[i]='Sell Lookback RSI Confirmed'
    
    # Reading trend of RSI for additional confirmation
    last_alert = None
    for i in range(lookback, len(prices)):
        if lookback_signals[i] == 'Buy Lookback RSI Confirmed':
            if last_alert != 'Buy' and rsi.iloc[i] > rsi.iloc[i-1]:
                print(f"Trend ALERT: RSI trending up after oversold on {prices.index[i].date()} at price {prices.iloc[i]:.2f}")
                last_alert = 'Buy'
        elif lookback_signals[i] == 'Sell Lookback RSI Confirmed':
            if last_alert != 'Sell' and rsi.iloc[i] < rsi.iloc[i-1]:
                print(f"Trend ALERT: RSI trending down after overbought on {prices.index[i].date()} at price {prices.iloc[i]:.2f}")
                last_alert = 'Sell'

    # Simulate trades
    position = 0
    cash = ending_cash
    portfolio_values = []
    buy_signals_dates = []
    buy_signals_prices = []
    sell_signals_dates = []
    sell_signals_prices = []
    buy_trades = 0
    sell_trades = 0
    profit = 0
    cost_basis = 0

    for i in range(len(prices)):
        price = prices.iloc[i]
        signal = signals[i]

        if signal == 'Buy' and position == 0:
            cost = price * shares_per_trade
            if cash >= cost:
                cash -= cost
                position = shares_per_trade
                cost_basis = price
                buy_trades += 1
                buy_signals_dates.append(prices.index[i])
                buy_signals_prices.append(price)
            else:
                # Not enough cash to buy
                signals[i] = 'Hold'

        elif signal == 'Sell' and position > 0:
            cash += price * position
            profit += (price - cost_basis) * position
            position = 0
            sell_trades += 1
            sell_signals_dates.append(prices.index[i])
            sell_signals_prices.append(price)

        # Track portfolio value each day
        portfolio_value = cash + position * price
        portfolio_values.append(portfolio_value)

    # Finalize if holding position at the end
    if position > 0:
        final_price = prices.iloc[-1]
        cash += final_price * position
        profit += (final_price - cost_basis) * position
        sell_signals_dates.append(prices.index[-1])
        sell_signals_prices.append(final_price)
        position = 0
        sell_trades += 1
        portfolio_values[-1] = cash
    total_realized_profit += profit
    ending_cash = cash

    roi_percent = (ending_cash - starting_cash) / starting_cash * 100

    all_results.append({
        'Ticker': ticker,
        'Buy Trades': buy_trades,
        'Sell Trades': sell_trades,
        'Ending Cash': cash,
        'Realized Profit': profit,
        'ROI (%)': roi_percent
    })

    # Plotting
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

    # --- Price and MAs ---
    ax1.plot(prices.index, prices, label='Price')
    ax1.plot(prices.index, short_ma, '--', label=f'Short {"EMA" if use_ema else "SMA"} ({short_window})')
    ax1.plot(prices.index, long_ma, '--', label=f'Long {"EMA" if use_ema else "SMA"} ({long_window})')
    ax1.scatter(buy_signals_dates, buy_signals_prices, marker='^', color='green', label='Buy Signal', s=100)
    ax1.scatter(sell_signals_dates, sell_signals_prices, marker='v', color='red', label='Sell Signal', s=100)
    ax1.set_title(f"{ticker} Crossover Strategy with Soft RSI Filter")
    ax1.set_ylabel("Price ($)")
    ax1.legend()
    ax1.grid(True)

    # --- RSI ---
    ax2.plot(prices.index, rsi, label= f'RSI ({rsiwindow})', color='purple')
    ax2.axhline(rsi_overbought, color='red', linestyle='--', label=f'Overbought ({rsi_overbought})')
    ax2.axhline(rsi_oversold, color='green', linestyle='--', label=f'Oversold ({rsi_oversold})')
    ax2.set_ylabel("RSI")
    ax2.set_ylim(0, 100)
    ax2.set_xlabel("Date")
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

    # Print trade summary for this ticker
    print(f"\nSummary of trades for {ticker}:")
    print(f"Buy Trades: {buy_trades}")
    print(f"Sell Trades: {sell_trades}")
    print(f"Ending Cash: ${cash:,.2f}")
    print(f"Realized Profit: ${profit:,.2f}")



# Run strategy for each ticker
for ticker in tickers:
    backtest_crossover(ticker)

# Final summary
results_df = pd.DataFrame(all_results)
print("\nAll strategies completed.")
print("\nFinal Summary for all tickers:")
print(results_df.to_string(index=False))
print(f"\nTotal Realized Profit Across All Tickers: ${total_realized_profit:,.2f}")
print(f"Ending Cash After All Trades: ${ending_cash:,.2f}")
print("Thank you for using Quantara!")

In [ ]:

# Calculate True Range (TR) and its smoothed version for ADX calculation more work need to be done here
#also need to know wtf i just did


#df['TR'] = pd.concat([df['high'] - df['low'], 
 #                     (df['high'] - df['close'].shift(1)).abs(), 
  #                    (df['low'] - df['close'].shift(1)).abs()], axis=1).max(axis=1)
#df['TR_smoothed'] = df['TR'].ewm(alpha=1/14, adjust=False).mean()
#df['+DI'] = 100 * (df['+DM_smoothed'] / df['TR_smoothed'])
#df['-DI'] = 100 * (df['-DM_smoothed'] / df['TR_smoothed'])
#df['DX'] = 100 * (df['+DI'] - df['-DI']).abs() / (df['+DI'] + df['-DI'])
#df['ADX'] = df['DX'].ewm(alpha=1/14, adjust=False).mean()
#ADX above 25 indicates a strong trend, while below 20 suggests a weak trend.
#The +DI and -DI lines can be used to generate buy/sell signals. A buy signal occurs when +DI crosses above -DI, and a sell signal occurs when -DI crosses above +DI.
#The ADX itself does not indicate trend direction, only trend strength.
# Consider integrating these indicators into the existing strategy for enhanced decision-making.
# Consider integrating these indicators into the existing strategy for enhanced decision-making.